# TechGuru Demo Notebook

**TechGuru** — Your AI Pair-Programmer That Actually Teaches You

This notebook demonstrates the core flows: code explanation, test generation, bug hunting, project scaffolding, running generated tests, and SRS scheduling. It uses the local `app.agent_core` functions and the demo FastAPI endpoints. If you want to run live Gemini calls on Kaggle, set the `GOOGLE_API_KEY` via Kaggle Secrets.

## 1) Install dependencies (optional)

Run this cell only if the environment is missing required packages. On Kaggle, set the `GOOGLE_API_KEY` in **Settings -> Secrets** before running cells that call Gemini.

In [ ]:

# Optional: install requirements (uncomment if needed)
# !pip install -r ../requirements.txt
# Alternatively install minimal packages
# !pip install google-genai fastapi pytest python-dotenv
print('Skip installation on Kaggle if packages are preinstalled.')


## 2) Import TechGuru agent (local package)
This notebook imports from the local `app` package (ensure you ran `pip install -e .` or your environment has the repo on PYTHONPATH).

In [ ]:

# Import core functions
from app import agent_core, code_tools, scaffolder, srs_scheduler
print('Imported agent_core:', agent_core)


## 3) Example: Explain code
We pass a small function to `explain_code()` and display the returned structured output. If your `GOOGLE_API_KEY` is set and a compatible model is available, this will call Gemini; otherwise it uses the offline fallback.

In [ ]:

code = """def add(a, b):
    return a + b
"""
res = agent_core.explain_code(code)
import json, textwrap, pprint
pprint.pprint(res)


## 4) Generate tests
Use `generate_tests` to create a pytest file. We'll write it to `demo/sample_project/tests/test_generated.py`. Then we'll run pytest on that folder.

In [ ]:

test_text = agent_core.generate_tests("""def add(a,b):\n    return a+b""", n_tests=4)
print('--- GENERATED TESTS ---\n')
print(test_text[:1000])
# write to demo/sample_project/tests/test_generated.py (create dirs if needed)
import os
out_dir = os.path.join('demo','sample_project','tests')
os.makedirs(out_dir, exist_ok=True)
with open(os.path.join(out_dir,'test_generated.py'),'w',encoding='utf-8') as f:
    f.write(test_text)
print('\nWrote tests to', os.path.join(out_dir,'test_generated.py'))


## 5) Scaffold a demo project and run tests
We'll scaffold a small project using `scaffold_project`, write files to `demo/sample_project`, and run pytest there. This demonstrates end-to-end behavior in the notebook environment.

In [ ]:

files = agent_core.scaffold_project('sample_project')
# write files under demo/sample_project/
import os
root = os.path.join('demo','sample_project')
for rel, content in files.items():
    parts = rel.split('/',1)
    path = parts[1] if len(parts)>1 else parts[0]
    full = os.path.join('demo','sample_project', path)
    os.makedirs(os.path.dirname(full), exist_ok=True)
    with open(full, 'w', encoding='utf-8') as f:
        f.write(content)
print('Scaffolded demo/sample_project with files:')
print('\n'.join(list(files.keys())))

# Run pytest in demo/sample_project
import subprocess, sys
proc = subprocess.run([sys.executable, '-m', 'pytest', '-q'], cwd=root, capture_output=True, text=True)
print('--- pytest output ---')
print(proc.stdout)
print(proc.stderr)


## 6) Bug hunting
Demonstrate `bug_hunt_and_fix` on a small, intentionally buggy snippet.

In [ ]:

buggy = """def divide(a, b):
    # naive division with no checks
    return a / b
"""
bh = agent_core.bug_hunt_and_fix(buggy)
import pprint
pprint.pprint(bh)


## 7) SRS Scheduler demo
Schedule a few micro-tasks and show the saved state (srs_state.json).

In [ ]:

r1 = srs_scheduler.schedule_task('unit-testing', quality=2)
r2 = srs_scheduler.schedule_task('edge-cases', quality=3)
import json, os
print('Scheduled tasks:')
print(json.dumps({'unit-testing': r1, 'edge-cases': r2}, indent=2))
# show state file if exists
if os.path.exists('srs_state.json'):
    print('\nSaved SRS state:')
    print(open('srs_state.json','r',encoding='utf-8').read())


## 8) Cost & Model Notes
This notebook uses environment variables `GOOGLE_API_KEY` and optional `GOOGLE_MODEL`/`GOOGLE_MODEL_DEEP` to control which Gemini models are used. On Kaggle, set the API key via **Settings -> Secrets**. Keep model choices light to save quota. The demo code falls back to offline stubs when no key is present.

## 9) Conclusion
This notebook demonstrates the core TechGuru flows: explanation, test generation, bug hunting, scaffolding, running tests, and scheduling learning tasks. For Kaggle submission: upload this notebook, set `GOOGLE_API_KEY` in notebook settings, and link your GitHub repo: https://github.com/shubhamranswal/TechGuru